In [1]:
import pandas as pd
import requests 
from bs4 import BeautifulSoup
import re


In [2]:
URLS = {
    "Ananea Castelo Suites Hotel": "https://m.tripadvisor.com/Hotel_Review-g189112-d33299137-Reviews-Castelo_Suites_Hotel_ananea-Albufeira_Faro_District_Algarve.html",
    "PortoBay Falésia": "https://www.tripadvisor.com/Hotel_Review-g1190872-d625806-Reviews-PortoBay_Falesia-Olhos_de_Agua_Albufeira_Faro_District_Algarve.html",
    "Regency Salgados Hotel & Spa": "https://www.tripadvisor.com/Hotel_Review-g11855863-d23418643-Reviews-Regency_Salgados_Hotel_Spa-Gale_Albufeira_Faro_District_Algarve.html",
    "NAU São Rafael Atlântico": "https://www.tripadvisor.com/Hotel_Review-g189112-d289104-Reviews-NAU_Sao_Rafael_Suites-Albufeira_Faro_District_Algarve.html",
    "NAU Salgados Dunas Suites": "https://www.tripadvisor.com/Hotel_Review-g775995-d1772673-Reviews-NAU_Salgados_Dunas_Suites-Guia_Albufeira_Faro_District_Algarve.html",
    "Vidamar Resort Hotel Algarve": "https://www.tripadvisor.com/Hotel_Review-g189112-d3927147-Reviews-VidaMar_Algarve-Albufeira_Faro_District_Algarve.html",
}

In [3]:
LOCATION_IDS ={'Ananea Castelo Suites Hotel': '33299137',
 'PortoBay Falésia': '625806',
 'Regency Salgados Hotel & Spa': '23418643',
 'NAU São Rafael Atlântico': '289104',
 'NAU Salgados Dunas Suites': '1772673',
 'Vidamar Resort Hotel Algarve': '3927147'}
    

In [4]:
import requests

TRIPADVISOR_API_KEY = "B55FDCE200304194AF4AEB1A264DB3DA"

# After you resolve them once, you store the location_ids here:
TRIPADVISOR_LOCATIONS = {
    # "Hotel Name": "location_id",
    # e.g. "Castelo Suites Hotel": "33299137",
}


In [5]:
import re

def ta_location_id_from_url(url: str) -> str | None:
    """
    Extract TripAdvisor location_id from a standard hotel URL.

    Example:
    https://www.tripadvisor.com/Hotel_Review-g189112-d33299137-Reviews-...
    -> "33299137"
    """
    if not url:
        return None
    m = re.search(r"-d(\d+)-", url)
    if not m:
        return None
    return m.group(1)


In [6]:
loc_id = dict()

In [7]:
for key in URLS:
    loc_id[key] = ta_location_id_from_url(URLS[key])
# Then manually add it to TRIPADVISOR_LOCATIONS:
# TRIPADVISOR_LOCATIONS["Castelo Suites Hotel"] = loc_id


In [8]:
loc_id

{'Ananea Castelo Suites Hotel': '33299137',
 'PortoBay Falésia': '625806',
 'Regency Salgados Hotel & Spa': '23418643',
 'NAU São Rafael Atlântico': '289104',
 'NAU Salgados Dunas Suites': '1772673',
 'Vidamar Resort Hotel Algarve': '3927147'}

In [9]:

def ta_get_rating(location_id: str):
    url = f"https://api.content.tripadvisor.com/api/v1/location/{location_id}/details"
    params = {
        "key": TRIPADVISOR_API_KEY,
        "language": "en",
    }
    resp = requests.get(url, params=params, timeout=15)
    print("details status:", resp.status_code)
    resp.raise_for_status()
    data = resp.json()
    print("details json:", data)

    rating_raw = data.get("rating")
    num_reviews = data.get("num_reviews") or data.get("review_count")
    try:
        rating = float(rating_raw) if rating_raw is not None else None
    except ValueError:
        rating = None

    print("Parsed rating:", rating, "num_reviews:", num_reviews)
    return rating, num_reviews

In [10]:
for key in loc_id:
    print(ta_get_rating(loc_id[key]))

details status: 200
details json: {'location_id': '33299137', 'name': 'Castelo Suites Hotel - ananea', 'web_url': 'https://www.tripadvisor.com/Hotel_Review-g189112-d33299137-Reviews-Castelo_Suites_Hotel_ananea-Albufeira_Faro_District_Algarve.html?m=66827', 'address_obj': {'street1': 'Estr. do Castelo', 'city': 'Albufeira', 'country': 'Portugal', 'postalcode': '8200-318', 'address_string': 'Estr. do Castelo, Albufeira 8200-318 Portugal'}, 'ancestors': [{'level': 'Municipality', 'name': 'Albufeira', 'location_id': '189112'}, {'level': 'District', 'name': 'Faro District', 'location_id': '3856572'}, {'level': 'Region', 'name': 'Algarve', 'location_id': '189111'}, {'level': 'Country', 'name': 'Portugal', 'location_id': '189100'}], 'latitude': '37.08045', 'longitude': '-8.298961', 'timezone': 'Europe/Lisbon', 'write_review': 'https://www.tripadvisor.com/UserReview-g189112-d33299137-Castelo_Suites_Hotel_ananea-Albufeira_Faro_District_Algarve.html?m=66827', 'ranking_data': {'geo_location_id': 